# Inference with LoRA Fine-tuned Qwen Model

This notebook demonstrates how to use your fine-tuned model for:
- Translation (EN ↔ RU)
- Question Answering (EN & RU)

In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

/home/shara/usi-nlp-EagleSFT/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [4]:
BASE_MODEL = "Qwen/Qwen2.5-0.5B"
LORA_ROOT = "./qwen2.5_lora"   # folder with checkpoints or final model
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float16

print(f"Device: {DEVICE}")

Device: cuda


## Load Model

In [5]:
def get_model_path(lora_root):
    """Get the path to the LoRA model (final > checkpoint > epoch)"""
    # Check for final model
    final_path = os.path.join(lora_root, "final")
    if os.path.exists(final_path):
        print(f"✅ Using final model: {final_path}")
        return final_path
    
    # Check for checkpoints
    checkpoints = [
        d for d in os.listdir(lora_root)
        if d.startswith("checkpoint-") and os.path.isdir(os.path.join(lora_root, d))
    ]
    if checkpoints:
        checkpoints.sort(key=lambda x: int(x.split("-")[1]))
        checkpoint_path = os.path.join(lora_root, checkpoints[-1])
        print(f"✅ Using latest checkpoint: {checkpoint_path}")
        return checkpoint_path
    
    # Check for epoch folders
    epochs = [
        d for d in os.listdir(lora_root)
        if d.startswith("epoch_") and os.path.isdir(os.path.join(lora_root, d))
    ]
    if epochs:
        epochs.sort(key=lambda x: int(x.split("_")[1]))
        epoch_path = os.path.join(lora_root, epochs[-1])
        print(f"✅ Using latest epoch: {epoch_path}")
        return epoch_path
    
    raise RuntimeError(f"No model found in {lora_root}")

LORA_PATH = get_model_path(LORA_ROOT)

✅ Using final model: ./qwen2.5_lora/final


In [6]:
print("🔤 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

🔤 Loading tokenizer...


In [7]:
print("🤖 Loading base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    dtype=DTYPE,
    device_map="auto",
    trust_remote_code=True
)

🤖 Loading base model...


In [8]:
print("🧩 Loading LoRA adapters...")
model = PeftModel.from_pretrained(
    base_model,
    LORA_PATH,
    torch_dtype=DTYPE
)
model.eval()

print("✅ Model loaded successfully!")

🧩 Loading LoRA adapters...
✅ Model loaded successfully!


## Inference Function

In [9]:
def generate_response(prompt, max_new_tokens=512, temperature=0.0, top_p=1.0):
    """
    Generate a response from the model.
    
    Args:
        prompt: The user's question or instruction
        max_new_tokens: Maximum tokens to generate
        temperature: Sampling temperature (0 = greedy/deterministic)
        top_p: Nucleus sampling parameter
    
    Returns:
        Generated response text
    """
    # Format the message
    message = f"<|user|>\n{prompt}\n<|assistant|>\n"
    
    # Tokenize
    inputs = tokenizer(
        message,
        return_tensors="pt",
        padding=False
    ).to(model.device)
    
    # Generate
    with torch.no_grad():
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens,
            do_sample=temperature > 0,
            temperature=temperature if temperature > 0 else None,
            top_p=top_p if temperature > 0 else None,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode only the generated part
    output_text = tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    print(f"Prompt: {prompt}")
    print(f"\nResponse: {output_text}")
    
    return output_text

## Example 1: Translation (Russian → English)

In [10]:
PROMPT = "Translate the following question to English:\n\nКогда была война?"

response = generate_response(PROMPT, max_new_tokens=128)

Prompt: Translate the following question to English:

Когда была война?

Response: When was the war?



## Example 2: Translation (English → Russian)

In [11]:
PROMPT = "Translate the following question to Russian:\n\nWhat is artificial intelligence?"

response = generate_response(PROMPT, max_new_tokens=128)

Prompt: Translate the following question to Russian:

What is artificial intelligence?

Response: Что такое искусственный интеллект?



## Example 3: Question Answering (English)

In [14]:
PROMPT = "What is machine learning?"

response = generate_response(PROMPT, max_new_tokens=256)

Prompt: What is machine learning?

Response: Machine learning is a subset of artificial intelligence that involves the development of algorithms and statistical models that enable computers to learn and improve from experience. It involves the use of statistical techniques to enable computers to automatically learn and improve from data, without being explicitly programmed to perform a specific task.

Here are some key aspects of machine learning:

1. **Data Collection**: Machine learning requires data to train models. This data can be in the form of text, images, audio, or any other type of data that can be processed by the algorithm.

2. **Model Training**: The algorithm learns from the data to identify patterns and make predictions or decisions. This is typically done using supervised learning, where the algorithm is given labeled data and learns to predict the output based on the input.

3. **Evaluation**: The performance of the model is evaluated using metrics such as accuracy, pr

## Example 4: Question Answering (Russian)

In [18]:
PROMPT = "Что такое машинное обучение?"

response = generate_response(PROMPT, max_new_tokens=256)

Prompt: Что такое машинное обучение?

Response: Машинное обучение — это процесс, направленный на обучение машинного интерфейса (машинного обучения) к определенным задачам, которые могут быть решены с помощью машинного обучения. Машинное обучение включает в себя разработку и внедрение алгоритмов машинного обучения для решения задач, таких как классификация, регрессия, регулирование и предсказание.

Основные этапы машинного обучения включают:

1. **Изучение задачи**: Определение задачи, которую нужно решить, и анализ ее сложности.
2. **Исследование алгоритмов**: Разработка и оценка различных алгоритмов машинного обучения, таких как линейный регрессион, градиентный спуск, случайные деревья и другие.
3. **Обучение алгоритмов**: Обучение алгоритмов машинного обучения на наборе данных, который содержит данные, которые нужно обучить машинному обучению.
4. **Проверка и оценка**: Проверка результатов обучения на тест


## Other Inferences

Test multiple prompts at once

In [20]:
test_prompts = [
    "Translate the following question to Russian:\n\nHow does photosynthesis work?",
    "Translate the following question to English:\n\nКак работает фотосинтез?",
    "What is the capital of Russia?",
    "Какая столица России?",
]

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n{'='*60}")
    print(f"Test {i}/{len(test_prompts)}")
    response = generate_response(prompt, max_new_tokens=256)


Test 1/4
Prompt: Translate the following question to Russian:

How does photosynthesis work?

Response: Как работает Photosynthesis?


Test 2/4
Prompt: Translate the following question to English:

Как работает фотосинтез?

Response: How does photo synthesis work?


Test 3/4
Prompt: What is the capital of Russia?

Response: The capital of Russia is Moscow. It is the largest city in Russia and the country's political, economic, and cultural center. Moscow is located in the northwestern part of the country and is the capital of the Russian Federation.


Test 4/4
Prompt: Какая столица России?

Response: Россия имеет несколько столиц, которые являются важными для экономики, политики и культурного наследия страны. Вот некоторые из них:

1. **Москва** — столица и центр экономики, политики и культуры. Она является основным городом и центром города, и является важным местом для культурного наследия, включая музеи, театры и другие культурные объекты.

2. **Санкт-Петербург** — столица и центр э